In [525]:
import numpy as np
import random
import Queue

In [526]:
class Vertex:
    
    n = 0
    
    def __init__(self):
        self.n = Vertex.n
        Vertex.n += 1
        
    def __repr__(self):
        return 'Vector ' + str(self.n)  
    

In [527]:
n_nodes = 5
n_edges = random.randint(n_nodes, n_nodes)
n_data = n_nodes/4

max_weight = 5


vertices = []
c_plus = {}
c_minus = {}
data_elements = {}

# create vertices
for n in xrange(n_nodes):
    vertices.append(Vertex())
    
# create edges
for n in xrange(n_edges):
    vi, vj = random.sample(vertices, 2)
    # sort vertices in edge to make it easier to compare two eges
    edge = tuple(sorted( (vi, vj) ))
    w = random.random() * max_weight
    
    # check if edge already exists
    if edge in c_plus or edge in c_minus:
        continue

    if random.random() > 0.5:
        c_plus[edge] = w
    else:
        c_minus[edge] = w
        
# add data elements    
for n in xrange(n_data):
    v = random.choice(vertices)
    w = random.random() * max_weight
    data_elements[v] = w


print 'positive constraints', c_plus
print 'negative constraints', c_minus
print 'data elements', data_elements


positive constraints {(Vector 4, Vector 0): 4.0315103959899075, (Vector 1, Vector 0): 3.746238234969628}
negative constraints {(Vector 4, Vector 2): 4.405052318732829, (Vector 1, Vector 3): 3.1944928079750263}
data elements {Vector 2: 2.813149371201153}


In [528]:
def generate_successors(state):
    vertices, true, false = state
    vertices = list(vertices) # copy list of vertices
    
    # no new states can be generated 
    if not vertices:
        return []
    
    v = vertices.pop()
    return [ ( vertices, true + [v], false ) , ( vertices, true, false + [v] ) ]

In [529]:
def evaluate(state, c_plus, c_minus, data_elements):
    vertices, true, false = state
    
    # not all elements have a true value yet
    if vertices:
        return 0
    
    coh = 0
    # consistent elements
    for (vi, vj), w in c_plus.items():
        if vi in true and vj in true:
            coh += w
    
    # inconsistent elements
    for (vi, vj), w in c_minus.items():
        if vi in true and vj in false or vi in false and vj in true:
            coh += w
            
    # data elements
    for v, w in data_elements.items():
        if v in c_plus:
            coh += w
            
    return coh
    
    

In [530]:
frontier = []
initial_state = (vertices, [], [])

# add initial state to frontier
frontier += generate_successors(initial_state)

best = (0, None)


while frontier:
    state = frontier.pop()
    coh = evaluate(state, c_plus, c_minus, data_elements)
    
    if coh > best[0]:
        _, true, false = state
        best = (coh, (true, false) )    
    
    frontier += generate_successors(state)
    
print 'Coherence Value:', best[0]
print 'True:', best[1][0]
print 'False:', best[1][1]

Coherence Value: 15.3772937577
True: [Vector 4, Vector 1, Vector 0]
False: [Vector 3, Vector 2]
